In [1]:
import pandas as pd
import anndata as ad
import scanpy as sc
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
import numpy as np

In [2]:
#Load the data
data = sc.read_h5ad('../../data/combined_data_5ht6hointersection_union_highly_var.h5ad')#Change path as required.
tfs = load_tf_names('../../data/allTFs_mm.txt')

ho6_data = data[data.obs['orig.ident'] == '6Ho']
ho6_basal = ho6_data[ho6_data.obs['cluster1'] == 'Mammary epithelial cells-Basal']
counts_df_6ho_basal = pd.DataFrame(ho6_basal.X, index=ho6_basal.obs_names,columns=ho6_basal.var_names)
tfs = sorted(list((set(tfs).intersection(ho6_basal.var_names))))

In [3]:
len(tfs)

451

# RUN GRNBOOST2 for all TFs 

In [ ]:
#Run grnboost2 with SGBM parameters, without a seed - 6ho_basal
for i in range(0,20):#This will run the loop 20 times:
    print(f'Run: {i}')
    #Run GRNBOOST2 for batches of 10 TFs.
    #Create a variable that will contain the entire edge list
    adjacencies = None
    index = 0
    iterations = np.floor(len(tfs)/10)
    n = 10
    count = 0
    #Run in batches of 10
    while count < iterations + 1:
        
        curr_tfs = tfs[index:index+n]
        curr_adjacencies = grnboost2(counts_df_6ho_basal, tf_names=curr_tfs, verbose=False)
        
        if adjacencies is None:
            adjacencies = curr_adjacencies
        else:
            adjacencies = pd.concat([adjacencies, curr_adjacencies], axis=0)
        index += 10
        count += 1
        if count == iterations:
            n = int(len(tfs) - iterations*n)
        print(f'Done for {count*10} TFs')
        if count%10 == 0:
            adjacencies.to_csv(f'../../results/results_step_1_grnboost2_6ho_basal/6Ho_basal_adjacencies_run{i}_{count*10}TFs.csv')
    adjacencies.to_csv(f'../../results/results_step_1_grnboost2_6ho_basal/6Ho_basal_adjacencies_run{i}_allTFs.csv')